In [ ]:
# En este notebook aprenderemos a:
# Leer los datos con PySpark
# Realizar transformaciones
# Guardar los dataframes como tablas Delta
# Crear un modelo estrella para análisis

#Configuración inicial y verificación del entorno
# Este comando verifica que estamos en el entorno correcto de Fabric

import pyspark.sql.functions as F
from pyspark.sql.types import *
from notebookutils import mssparkutils
import pandas as pd

# Verificar versión de Spark
print(f"Versión de Spark: {spark.version}")

### Tabla Brokers

In [ ]:
df_brokers = spark.read.parquet("abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/brokers_delta")
# df now is a Spark DataFrame containing parquet data from "abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/brokers_delta/part-00000-3a3a0989-8d0f-40a5-8d1a-fa23748f975a-c000.snappy.parquet".
display(df_brokers)

In [ ]:
df_brokers.write.format("delta").mode("overwrite").saveAsTable("brokers")

###

### Tabla Campaigns

In [ ]:
df_campaigns = spark.read.parquet("abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/campaigns_delta")
# df now is a Spark DataFrame containing parquet data from "abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/brokers_delta/part-00000-3a3a0989-8d0f-40a5-8d1a-fa23748f975a-c000.snappy.parquet".
#display(df_campaigns)

In [ ]:
df_campaigns = df_campaigns.select("campaignID","Channel","CampaignName","StartDate","EndDate","BudgetUSD")
display(df_campaigns.limit(5))

In [ ]:
df_campaigns.write.format("delta").mode("overwrite").saveAsTable("campaigns")

### tabla projects

In [ ]:
df_projects = spark.read.parquet("abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/projects_delta")
# df now is a Spark DataFrame containing parquet data from "abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/brokers_delta/part-00000-3a3a0989-8d0f-40a5-8d1a-fa23748f975a-c000.snappy.parquet".
#display(df_campaigns)

In [ ]:
display(df_projects.limit(5))

In [ ]:
df_projects.write.format("delta").mode("overwrite").saveAsTable("projects")

### Tabla Properties

In [ ]:
df_properties = spark.read.parquet("abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/properties_delta")

In [ ]:
display(df_properties.limit(10))

In [ ]:
df_properties.write.format("delta").mode("overwrite").saveAsTable("properties")

### Tabla Clientes

In [ ]:
df_clientes = spark.read.parquet("abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/clients_delta")

In [ ]:
df_clientes.write.format("delta").mode("overwrite").saveAsTable("clients")

### Tabla LEADS

In [ ]:
df_leads = spark.read.parquet("abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/leads_delta")

In [ ]:
df_leads.createOrReplaceTempView("leads")

In [ ]:
df_leads_procesado = spark.sql("""
SELECT 
l.LeadID,
l.ClientID,
l.PropertyID,
p.ListPriceUSD,
l.CampaignID,
COALESCE(c.BudgetUSD,0) AS BudgetUSD,
l.LeadDate,
l.LeadSource

FROM leads l 
left join LKH_PROY_INTEGRADOR_RCALL.properties p
on 
l.PropertyID = p.PropertyID

left join LKH_PROY_INTEGRADOR_RCALL.campaigns c
on 
l.CampaignID = c.CampaignID AND
l.LeadDate BETWEEN c.StartDate and c.EndDate
""")
display(df_leads_procesado)

In [ ]:

df_leads_procesado = df_leads_procesado.withColumn(
    "idLeadDate",
    F.date_format(F.col("LeadDate"), "yyyyMMdd").cast("int")
)

In [ ]:
display(df_leads_procesado)


In [ ]:
df_leads_procesado.write.format("delta").mode("overwrite").saveAsTable("leads")

### tabla Sales

In [ ]:
df_sales = spark.read.parquet("abfss://WSP_ProyectoIntegrador_RCLL@onelake.dfs.fabric.microsoft.com/LKH_PROY_INTEGRADOR_RCALL.Lakehouse/Files/Silver/Sales_delta")

In [ ]:
df_sales.createOrReplaceTempView("sales")

In [ ]:
df_sales_procesado = spark.sql("""
SELECT 
s.SaleID,
s.ClientID,
s.PropertyID,
p.ListPriceUSD,
s.BrokerID,
s.SaleDate,
s.SalePriceUSD
FROM sales s 
INNER join LKH_PROY_INTEGRADOR_RCALL.properties p
on 
S.PropertyID = p.PropertyID


""")
display(df_sales_procesado.limit(10))

In [ ]:

df_sales_procesado = df_sales_procesado.withColumn(
    "idSaleDate",
    F.date_format(F.col("SaleDate"), "yyyyMMdd").cast("int")
)

In [ ]:
df_sales_procesado.write.format("delta").mode("overwrite").saveAsTable("sales")